In [2]:
import numpy as np
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os

def calculate_profiles(img):
    return {
        'x': np.sum(img, axis=0).astype(int),
        'y': np.sum(img, axis=1).astype(int)
    }

def get_vertical_boxes(img):
    profiles = calculate_profiles(img)
    up = 0
    down = img.shape[0] - 1

    while up < len(profiles['y']) and profiles['y'][up] == 0:
        up += 1
    
    while down >= 0 and profiles['y'][down] == 0:
        down -= 1

    return up, down + 1

def get_horizontal_boxes(img):
    profiles = calculate_profiles(img)
    borders = []
    min_pixels = 0

    i = 0
    while i < profiles['x'].shape[0]:
        current = profiles['x'][i]
        if current > min_pixels:
            x1 = i
            while i < profiles['x'].shape[0] and profiles['x'][i] > min_pixels:
                i += 1
            borders.append((x1, i))
        i += 1

    return borders

def is_enye(character_img):
    template_path = 'alphabet_images/Ñ.png'  
    
    try:
        template = cv2.imread(template_path, 0)
        if template is None:
            raise FileNotFoundError(f"Failed to load template image: {template_path}")
        
        char_h, char_w = character_img.shape
        tmpl_h, tmpl_w = template.shape
        
        if tmpl_h != char_h or tmpl_w != char_w:
            template = cv2.resize(template, (char_w, char_h))
        
        result = cv2.matchTemplate(character_img, template, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8  
        
        loc = np.where(result >= threshold)
        
        if len(loc[0]) > 0:
            return True
        else:
            return False
    
    except Exception as e:
        print(f"Error loading template image: {e}")
        return False

def visualize_segments(image_path, segments):
    image = Image.open(image_path).convert('L')
    draw = ImageDraw.Draw(image)
    
    for (x1, y1, x2, y2) in segments:
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    
    image.show()

def plot_character_profiles(alphabet_images_dir):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZÑ'
    
    for char in alphabet:
        image_path = os.path.join(alphabet_images_dir, f'{char}.png')
        if not os.path.exists(image_path):
            print(f"Image for character {char} not found.")
            continue
        img = np.array(Image.open(image_path).convert('L'))
        profile_x = calculate_profiles(img)['x']
        profile_y = calculate_profiles(img)['y']
        
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.bar(range(len(profile_x)), profile_x)
        plt.title(f'X Profile of {char}')
        plt.xlabel('X')
        plt.ylabel('Sum of Black Pixels')
        
        plt.subplot(1, 2, 2)
        plt.bar(range(len(profile_y)), profile_y)
        plt.title(f'Y Profile of {char}')
        plt.xlabel('Y')
        plt.ylabel('Sum of Black Pixels')
        
        plt.tight_layout()
        profiles_dir = 'profiles'
        os.makedirs(profiles_dir, exist_ok=True)
        plt.savefig(os.path.join(profiles_dir, f'{char}_profile.png'))
        plt.close()
        print(f"Profiles saved for {char}")

def main():
    image_path = 'img.bmp'
    
    img_src = Image.open(image_path).convert('L')
    data = np.array(img_src)
    _, binary_img = cv2.threshold(data, 128, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((2, 2), np.uint8)
    eroded = cv2.erode(binary_img, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=2)

    character_boundaries = []
    for (x1, x2) in get_horizontal_boxes(dilated):
        y1, y2 = get_vertical_boxes(dilated[:, x1:x2])
        character_boundaries.append((x1, y1, x2, y2))
    
    print(f"Character boundaries: {character_boundaries}")
    
    for (x1, y1, x2, y2) in character_boundaries:
        char_img = data[y1:y2, x1:x2]
        if is_enye(char_img):
            print(f"Character at ({x1}, {y1}, {x2}, {y2}) is Ñ")
        else:
            print(f"Character at ({x1}, {y1}, {x2}, {y2}) is not Ñ")
    
    visualize_segments(image_path, character_boundaries)
    
    alphabet_images_dir = 'alphabet_images'
    plot_character_profiles(alphabet_images_dir)

if __name__ == '__main__':
    main()


Character boundaries: [(26, 186, 52, 221), (58, 186, 84, 221), (103, 186, 112, 221), (117, 186, 149, 221), (157, 186, 180, 221), (186, 186, 196, 221), (203, 186, 229, 221), (235, 186, 265, 221), (268, 186, 300, 221), (306, 185, 340, 222), (359, 186, 385, 221), (390, 185, 414, 222), (418, 186, 447, 221), (450, 186, 484, 221), (502, 186, 535, 221), (539, 186, 573, 221), (577, 185, 608, 222), (614, 186, 624, 221), (629, 185, 663, 222), (680, 186, 709, 221), (712, 185, 746, 222), (752, 186, 784, 221), (788, 185, 822, 222), (827, 185, 851, 222), (870, 186, 896, 221), (900, 185, 934, 222), (939, 185, 963, 222), (982, 186, 1014, 221), (1020, 186, 1046, 221), (1052, 186, 1091, 221), (1096, 185, 1130, 222), (1134, 186, 1166, 221), (1175, 186, 1184, 221), (1189, 185, 1223, 222), (1228, 185, 1252, 222), (1271, 186, 1297, 221), (1302, 185, 1326, 222), (1330, 186, 1359, 221), (1362, 186, 1396, 221), (1400, 175, 1432, 221), (1451, 186, 1485, 221), (1489, 185, 1523, 230), (1529, 186, 1560, 222), (156